In [3]:
import pandas as pd

# Read data from files
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
test = pd.read_csv('testData.tsv', header=0, delimiter='\t', quoting=3)
unlabeled_train = pd.read_csv('unlabeledTrainData.tsv', header=0, delimiter='\t', quoting=3)

# Verify the numer of reviews that were read (100, 00 in total)
print("""Read {} labeled train reviews, 
{} labeled test reviews, 
and {} unlabeled reviews""".format(
    train['review'].size, test['review'].size, unlabeled_train['review'].size))

Read 25000 labeled train reviews, 
        25000 labeled test reviews, 
        and 50000 unlabeled reviews


In [4]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    """Function to convert a  document to a sequence of words, optionally removing stop words. Returns a list of words."""
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 2. Remove Non-letters
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    # 4. Optionally remove stop words
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [word for word in words if not word in stops]
    return words
        

In [8]:
# Downlaod the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function ot split a review into parsed sentences
def review_to_sentences(review, tokenzier, remove_stopwords=False):
    """Function to split a review into parsed sentences. Returns
    a lsit of sentences, where each sentence is a list of words"""
    # 1. User the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        #If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists)
    return sentences

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [10]:
sentences = [] # Initialize an empty list of sentences

print('Parsing sentences from training set')
for review in train['review']:
    sentences += review_to_sentences(review, tokenizer)
    
print('Parsing sentences from unlabled set')
for review in unlabeled_train['review']:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:198: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.happierabroad.com"'" looks like a URL. Beautiful So

Parsing sentences from unlabled set


/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.archive.org/details/LovefromaStranger"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/memery/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.loosechangeguide.com/LooseChangeGuide.html"'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful So

In [11]:
print(len(sentences))
print(sentences[0])
print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [15]:
# Import the built-in logging module and configure it so that Word2Vec
# creates nice ouput messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300 # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4 # Number of threads to run in parallel
context = 10 # Context window size
downsampling = 1e-3 # Downsample setting for frequence words

# Initialize and train model (this will take some time)
from gensim.models import word2vec
print('Training model...')
model = word2vec.Word2Vec(sentences,
                         workers=num_workers,
                         size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling)
# If you don't plan to train the model any further, calling
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)
# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = '300features_40minwords_10context'
model.save(model_name)

Training model...


In [16]:
print(model.doesnt_match('man woman child kitchen'.split()))

kitchen


In [17]:
print(model.doesnt_match('france england germany berlin'.split()))

berlin


In [18]:
print(model.doesnt_match('paris berlin london austria'.split()))

paris


In [20]:
print(model.most_similar('man'))
print(model.most_similar('queen'))

[('woman', 0.620460033416748), ('lady', 0.5978381633758545), ('lad', 0.5947127342224121), ('millionaire', 0.5336859226226807), ('guy', 0.5209099054336548), ('monk', 0.5120577812194824), ('men', 0.5044459104537964), ('soldier', 0.5028200149536133), ('boy', 0.49186182022094727), ('person', 0.49071723222732544)]
[('princess', 0.6938101649284363), ('bride', 0.6378064155578613), ('maid', 0.5931621789932251), ('belle', 0.5840067267417908), ('stepmother', 0.5785546898841858), ('mistress', 0.5769913792610168), ('nun', 0.5756708383560181), ('mary', 0.5738168954849243), ('goddess', 0.5737065076828003), ('victoria', 0.5731890201568604)]


In [21]:
print(model.most_similar('awful'))

[('terrible', 0.7707134485244751), ('horrible', 0.7411921620368958), ('atrocious', 0.7342128753662109), ('abysmal', 0.7259578704833984), ('dreadful', 0.703248918056488), ('appalling', 0.6884777545928955), ('horrendous', 0.6792418956756592), ('horrid', 0.6619541049003601), ('lousy', 0.6417896747589111), ('amateurish', 0.6281046867370605)]
